### LSTM model tuner

* Define functions
* try to find the best hyperparameters
* Train the model

Importing libraries

In [139]:
import pandas as pd
from matplotlib import pyplot

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Input
from keras.callbacks import TensorBoard
from keras.layers import Dropout
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

from math import sqrt
from numpy import concatenate

Define functions

In [140]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [141]:
tf.compat.v1.keras.layers.CuDNNLSTM

def lstm_model_builder(hp):

    model = Sequential()

    # model.add(Input(
    #     shape=(train_x.shape[1],train_x.shape[2])
    # ))
    
    model.add(LSTM(hp.Int('input_unit',
                        min_value=32,
                        max_value=512,
                        step=32), 
                        return_sequences=True, 
                        input_shape=(train_x.shape[1],train_x.shape[2])
                        ))

    for i in range(hp.Int('n_layers', 1, 3)):

        model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
        model.add(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=512,step=32),return_sequences=True))
        model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
        model.add(Dense(train_y.shape[1], activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))
        model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['mse'])
        
    return model

Load data

In [142]:
df = pd.read_parquet("../data/processed/new-avc.parquert")
df.head()

,date,co_ppb,o3_ppb,pm25_ugm3,so2_ugm3,temperatura_c,umidade_relativa_percentual,precipitacao_mmdia,focos_queimada,respiratory,avc
0,2013-01-01,135.40,11.125,8.175,0.875,26.450,81.75,2.0,0.0,6.0,1.0
1,2013-01-02,155.05,10.925,11.675,1.000,26.400,82.00,2.0,0.0,8.0,0.0
2,2013-01-03,137.40,12.225,8.800,0.825,27.400,81.25,0.0,0.0,12.0,1.0
3,2013-01-04,128.30,13.750,11.650,0.925,27.750,76.25,1.0,0.0,9.0,0.0
4,2013-01-05,128.25,10.950,7.800,1.075,26.625,84.75,9.0,0.0,10.0,1.0


Normalize features

In [143]:
df = df.set_index("date")

In [144]:
df = df.drop(columns=["avc"])

In [145]:
values = df.values

In [146]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

Apply series_to_supervised

In [147]:
n_lags = 3

In [148]:
df_sl = series_to_supervised(scaled, n_lags, 1)

Drop columns I don't want to predict

In [149]:
df_sl.drop(df_sl.columns[[27, 28, 29, 30, 31, 32, 33, 34]], axis=1, inplace=True)

Define and fit model

Split into train and test sets

In [150]:
values = df_sl.values
train_set, test_set = train_test_split(
    df_sl, test_size=0.2, shuffle=False
)

In [151]:
train_X, train_Y = train_set.drop("var9(t)", axis=1), train_set.drop(train_set.columns[:27], axis=1)
test_X, test_Y = test_set.drop("var9(t)", axis=1), test_set.drop(test_set.columns[:27], axis=1)

Reshape input to be 3D [samples, timesteps, features]

In [152]:
train_x, test_x = train_X.values, test_X.values
train_y, test_y = train_Y.values, test_Y.values

In [153]:
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

In [154]:
tuner= RandomSearch(
        lstm_model_builder,
        objective='mse',
        max_trials=2,
        executions_per_trial=1
        )

In [155]:
tuner.search(
        x=train_x,
        y=train_y,
        epochs=20,
        batch_size=128,
        validation_data=(test_x,test_y),
)

Trial 2 Complete [00h 00m 12s]
mse: 0.017918679863214493

Best mse So Far: 0.017918679863214493
Total elapsed time: 00h 00m 37s
INFO:tensorflow:Oracle triggered exit


In [156]:
best_model = tuner.get_best_models(num_models=1)[0]

In [157]:
yhat = best_model.predict(test_x)
test_x = test_x.reshape((test_x.shape[0], test_x.shape[2]))
yhat = yhat.reshape((yhat.shape[0], yhat.shape[1]))

14/14 [==============================] - 1s 6ms/step


In [158]:
# invert scaling for forecast
inv_yhat = concatenate((test_x[:, -8:], yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,8]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_x[:, -8:], test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,8]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 3.628


In [159]:
pd.DataFrame(test_x[:, -8:])

,0,1,2,3,4,5,6,7
0,0.334824,0.089580,0.148235,0.991239,0.753788,0.000000,0.00,0.285714
1,0.260308,0.113697,0.188235,0.750939,0.882576,0.160000,0.00,0.250000
2,0.281669,0.090840,0.221176,0.717146,0.928030,0.000000,0.08,0.142857
3,0.358172,0.099160,0.167059,0.619524,0.954545,0.320000,0.00,0.178571
4,0.314953,0.090168,0.160000,0.622028,0.909091,0.000000,0.00,0.214286
...,...,...,...,...,...,...,...,...
433,0.109290,0.064034,0.120000,0.694618,0.969697,0.000000,0.00,0.000000
434,0.098361,0.050756,0.115294,0.664581,0.969697,0.000000,0.00,0.000000
435,0.133135,0.038992,0.040000,0.664581,0.965909,0.373333,0.00,0.000000
436,0.135122,0.038655,0.030588,0.649562,0.965909,0.026667,0.00,0.000000
